# ![xframe](images/xframe.png)


<center>N-dimensional dataframe with lazy evaluation</center>

<center> <h1>xframe is a dataframe for C++, based on xtensor and xtl</h1> </center>

In [ ]:
#include <string>
#include <iostream>

#include "xtl/xbasic_fixed_string.hpp"

#include "xtensor/xio.hpp"
#include "xtensor/xrandom.hpp"
#include "xtensor/xmath.hpp"

#include "xframe/xio.hpp"
#include "xframe/xvariable.hpp"
#include "xframe/xvariable_view.hpp"
#include "xframe/xvariable_masked_view.hpp"
#include "xframe/xio.hpp"

In [ ]:
using fstring = xtl::xfixed_string<55>;

using data_type = xt::xoptional_assembly<xt::xarray<double>, xt::xarray<bool>>;
using coordinate_type = xf::xcoordinate<fstring, data_type::size_type>;

using variable_type = xf::xvariable<coordinate_type, data_type>;

# Variable with 2D Data

## Dry temperature variable

In [ ]:
data_type dry_temperature_data = xt::eval(xt::random::rand({6, 3}, 15., 25.));
dry_temperature_data(0, 0).has_value() = false;
dry_temperature_data(2, 1).has_value() = false;

In [ ]:
dry_temperature_data

In [ ]:
auto time_axis = xf::axis({"2018-01-01", "2018-01-02", "2018-01-03", "2018-01-04", "2018-01-05", "2018-01-06"});

In [ ]:
auto dry_temperature = variable_type(
    dry_temperature_data,
    {
        {"date", time_axis},
        {"city", xf::axis({"London", "Paris", "Brussels"})}
    }
);

In [ ]:
dry_temperature

In [ ]:
dry_temperature.select({{"city", "London"}, {"date", "2018-01-04"}})

In [ ]:
dry_temperature.locate("2018-01-03", "Brussels")

In [ ]:
dry_temperature.locate("2018-01-03", "Paris")

## Relative humidity variable

In [ ]:
data_type relative_humidity_data = xt::eval(xt::random::rand({3}, 50.0, 70.0));

auto relative_humidity = variable_type(
    relative_humidity_data,
    {
        {"city", xf::axis({"Paris", "London", "Brussels"})}
    }
);

relative_humidity

## Compute water vapour pressure using Broadcasting

In [ ]:
dry_temperature

In [ ]:
auto water_vapour_pressure = 0.01 * relative_humidity * 6.1 * xt::exp((17.27 * dry_temperature) / (237.7 + dry_temperature));

In [ ]:
water_vapour_pressure

## Variable with 3D data

In [ ]:
data_type pressure_data = {{{ 1.,  2., 3. },
                            { 4.,  5., 6. },
                            { 7.,  8., 9. }},
                           {{ 1.3, 1.5, 1.},
                            { 2., 2.3, 2.4},
                            { 3.1, 3.8, 3.}},
                           {{ 8.5, 8.2, 8.6},
                            { 7.5, 8.6, 9.7},
                            { 4.5, 4.4, 4.3}}};

In [ ]:
auto pressure = variable_type(
    pressure_data,
    {
        {"x", xf::axis(3)},
        {"y", xf::axis(3, 6, 1)},
        {"z", xf::axis(3)},
    }
);

In [ ]:
pressure

In [ ]:
pressure.select({{"x", 1}, {"y", 4}, {"z", 1}})

In [ ]:
pressure.locate(0, 5, 2)

## Masking

In [ ]:
auto masked_pressure = xf::where(
    pressure,
    not_equal(pressure.axis<int>("x"), 2) && pressure.axis<int>("z") < 2
);

In [ ]:
masked_pressure